##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Install / Import / Load

This code depends on [Lucid](https://github.com/tensorflow/lucid) (our visualization library), and [svelte](https://svelte.technology/) (a web framework). The following cell will install both of them, and dependancies such as TensorFlow. And then import them as appropriate.

In [2]:
import numpy as np
import tensorflow as tf
from skimage.transform import resize
from skimage.io import imsave
from os import getcwd, listdir
from PIL import Image

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
from lucid.misc.gradient_override import gradient_override_map
import lucid.scratch.web.svelte as lucid_svelte

# import sys
# sys.path.append('../')
# from top5gen.examples.inception_pretrained import get_top_five

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets

# Attribution Code

In [3]:
# # This, obviously, loads up the model
# model = models.InceptionV1()
# model.load_graphdef()

tf.logging.set_verbosity(tf.logging.ERROR)
sess = tf.InteractiveSession()
image = tf.Variable(tf.zeros((224, 224, 3)))

def inception(image, reuse):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v1_arg_scope(weight_decay=0.0)
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v1(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
    return logits, probs

logits, probs = inception(image, reuse=False)

In [4]:
import tempfile
from urllib.request import urlretrieve
import tarfile
import os

data_dir = tempfile.mkdtemp()
inception_tarball, _ = urlretrieve(
    'http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz')
tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)

restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV1/')
]
saver = tf.train.Saver(restore_vars)
saver.restore(sess, os.path.join(data_dir, 'inception_v1.ckpt'))

In [5]:
import json
import matplotlib.pyplot as plt

imagenet_json, _ = urlretrieve(
    'http://www.anishathalye.com/media/2017/07/25/imagenet.json')
with open(imagenet_json) as f:
    imagenet_labels = json.load(f)
    
def get_top_five(path, img):
    # Get predictions
    p = sess.run(probs, feed_dict={image: img})[0]
    topk = list(p.argsort()[-10:][::-1])
    topprobs = p[topk]
    
    # Put results in nice list
    current_results = [path]
    for i in range(5):
        current_results.append('{}: probability: {:.02f}, label: {}'
              .format(i+1, topprobs[i], imagenet_labels[topk[i]]))

    #return(topk, topprobs, [imagenet_labels[i] for i in topk])
    # Legacy requires us to wrap the results in another empty list
    to_return = []
    to_return.append(current_results)
    return(to_return)

In [6]:
# import PIL
# import numpy as np

# img_path, _ = urlretrieve('http://www.anishathalye.com/media/2017/07/25/cat.jpg') #urlretrieve('file:///home/w266ajh/Documents/jupyter_notebooks/adslk/adv.png')
# img_class = 281
# img = PIL.Image.open(img_path)
# big_dim = max(img.width, img.height)
# wide = img.width > img.height
# new_w = 224 if not wide else int(img.width * 224 / img.height)
# new_h = 224 if wide else int(img.height * 224 / img.width)
# img = img.resize((new_w, new_h)).crop((0, 0, 224, 224))
# img = (np.asarray(img) / 255.0).astype(np.float32)
# with open("adv_input", "w") as out:
#     out.write(str(img))
    
# a = get_top_five(img)
# for thing in a: 
#     for t in thing: print(t)

TypeError: get_top_five() missing 1 required positional argument: 'img'

In [8]:
# Here we take in the 1000-dimension vector of potential image labels
labels_str = read("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt")
labels = [line[line.find(" "):].strip().encode() for line in labels_str.decode().split("\n")]
labels = [label.decode().strip().split()[1].replace("_", " ") for label in labels]
labels = ["dummy"] + labels

In [9]:
model = models.InceptionV1()
model.load_graphdef()

def raw_class_spatial_attr(img, layer, label, override=None):
  """How much did spatial positions at a given layer effect a output class?"""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
    
    # display results

    
    if label is None: return np.zeros(acts.shape[1:-1])
    
    # Compute gradient
    softmax_result = T("softmax2_pre_activation")
    score = softmax_result[0, labels.index(label)] # The "score" is the softmax result at idx [0, label idx]
    t_grad = tf.gradients([score], [T(layer)])[0]   
    grad = t_grad.eval({T(layer) : acts})
    
    # Linear approximation of effect of spatial position
    return np.sum(acts * grad, -1)[0]

In [10]:
def raw_spatial_spatial_attr(img, layer1, layer2, override=None):
  """Attribution between spatial positions in two different layers."""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts1 = T(layer1).eval()
    acts2 = T(layer2).eval({T(layer1) : acts1})
    
    # Construct gradient tensor
    # Backprop from spatial position (n_x, n_y) in layer2 to layer1.
    n_x, n_y = tf.placeholder("int32", []), tf.placeholder("int32", [])
    layer2_mags = tf.sqrt(tf.reduce_sum(T(layer2)**2, -1))[0]
    score = layer2_mags[n_x, n_y]
    t_grad = tf.gradients([score], [T(layer1)])[0]
    
    # Compute attribution backwards from each positin in layer2
    attrs = []
    for i in range(acts2.shape[1]):
      attrs_ = []
      for j in range(acts2.shape[2]):
        grad = t_grad.eval({n_x : i, n_y : j, T(layer1) : acts1})
        # linear approximation of imapct
        attr = np.sum(acts1 * grad, -1)[0]
        attrs_.append(attr)
      attrs.append(attrs_)
  return np.asarray(attrs)

# Spatial Attribution Interface

In this section, we build the *interface* for interacting with the different kinds of spatial attribution data that we can compute using the above functions. Feel free to skip over this if you aren't interested in that part. The main reason we're including it is so that you can change the interface if you want to.


In [11]:
%%html_define_svelte SpatialWidget

<div class="figure" style="width: 10000px; height: 250px; contain: strict;">
  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint1 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed3a {{pct1}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint2 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed3b {{pct2}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint3 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4a {{pct3}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint4 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4b {{pct4}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint5 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4c {{pct5}}</div>
  </div>

 <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint6 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4d {{pct6}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint6 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4e {{pct7}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint8 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed5a {{pct8}}</div>
  </div>

    <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint9 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed5b {{pct9}}</div>
  </div>
  
</div>


<style>

  .outer{
    width: 224px;
    height: 224px;
    display: inline-block;
    margin-right: 2px;
    position: relative;
  }
  .outer img, .outer svg {
    position: absolute;
    left: 0px;
    top: 0px;
    width: 224px;
    height: 224px;
    image-rendering: pixelated; 
  }
  .attr {
    opacity: 0.6;
  }
  .pointer_container {
    z-index: 100;
  }
  .pointer_container rect {
    opacity: 0;
  }
  .pointer_container .selected  {
    opacity: 1;
    fill: none;
    stroke: hsl(24, 100%, 50%);
    stroke-width: 0.1px;
  }
  .label{
    position: absolute;
    left: 0px;
    top: 224px;
    width: 224px;
  }
</style>

<script>
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        img: "",
        hint1: "",
        hint2: "",
        hint3: "",
        hint4: "",
        hint5: "",
        hint6: "",
        hint7: "",
        hint8: "",
        hint9: "",
        pct1: "",
        pct2: "",
        pct3: "",
        pct4: "",
        pct5: "",
        pct6: "",
        pct7: "",
        pct8: "",
        pct9: "",
        spritemap1 : "",
        size1: 1,
        spritemap2 : "",
        size2: 1,
        pos1: undefined,
        pos2: undefined,
        layer1: "",
        layer2: ""
      };
    },
    computed: {
      xs1: (size1) => range(size1),
      ys1: (size1) => range(size1),
      xs2: (size2) => range(size2),
      ys2: (size2) => range(size2)
    },
    helpers: {range}
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_ruc1dwot/SpatialWidget_00950ed4_270e_495f_b8b0_acfa1c5a92f1.html > /tmp/svelte_ruc1dwot/SpatialWidget_00950ed4_270e_495f_b8b0_acfa1c5a92f1.js
b'svelte version 1.64.1\ncompiling ../../../../tmp/svelte_ruc1dwot/SpatialWidget_00950ed4_270e_495f_b8b0_acfa1c5a92f1.html...\n(4:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(5:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(11:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(12:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(18:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(19:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(25:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(26:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(32:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(33:4) \xe2\x80\

In [12]:
def orange_blue(a,b,clip=False):
  """This is what makes the orange/blue highlights in the saliency map"""
  
  # Clip *is* called on each run in the given code
  # --This appears to cut down on noise (in purple) in the final output
  if clip:
    a,b = np.maximum(a,0), np.maximum(b,0)
    
  # This is all original to what I was given, I don't know what the constants represent
  img = np.stack([a, (a + b)/2., b], -1)
  img /= 1e-2 + np.abs(img).max()/1.5
  img += 0.3  # Brightens the image a tad
  img[img>1] = 1 # Truncate oddities
  highlight_count = 0
  pixel_count = 0
  
  # Gets the number of pixels that aren't black
  # Due to some image brightening, 'black' is considered 0.3
  for row in img:
    for pixel in row:
        pixel_count += 1
        for rgbval in pixel:
          if rgbval > 0.3:
            highlight_count += 1
            break
  
  return [img, highlight_count/pixel_count]

# Simple Attribution Example

In [13]:
def image_url_grid(grid):
  return [[_image_url(img) for img in line] for line in grid ]

In [29]:
# MULTIPLE INPUT DIRS

# This block gets the saliency maps themselves

layers = ['mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e', 'mixed5a', 'mixed5b']

# Use for single input dir
# cur_labels = get_top_five("/home/w266ajh/Documents/top5gen/data_imageparts", ".jpg")

# Use for multiple input dirs
# Root dir
root_dir = "/home/w266ajh/Documents/top5gen/similar-images/"

# Go thorugh each dir in root dir
for dir in listdir(root_dir):
    go = False
    print(dir)
    # Continue if dir is empty
    if listdir(root_dir + dir) == list(): continue
    # Continue if no jpgs
    for file in listdir(root_dir + dir):
        if file.endswith(".jpg"): go = True
    if not go: continue
    # Manually iterate across image dir
    for file in listdir(root_dir + dir):
        if file.endswith(".jpg"):
            img = PIL.Image.open(root_dir + dir + "/" + file)
            big_dim = max(img.width, img.height)
            wide = img.width > img.height
            new_w = 224 if not wide else int(img.width * 224 / img.height)
            new_h = 224 if wide else int(img.height * 224 / img.width)
            img = img.resize((new_w, new_h)).crop((0, 0, 224, 224))
            img = (np.asarray(img) / 255.0).astype(np.float32)
            cur_labels = get_top_five(root_dir + dir + "/" + file, img)
        else: continue
        for label in cur_labels:
            # Note: Must save the file off each time because it will not re-read raw ndarray
            file = label.pop(0)
            print(file)
            img = resize(load(file), (224, 224))  # Make all 224x224
            imsave("current.jpeg", img, plugin='pil', format_str='jpeg')
            img = load("current.jpeg")
            for tag in label:
                print(tag)
                saliency_layers = []
                saliency_values = []
                for layer in layers:
                    # Get our saliency layer
                    saliency = orange_blue(
                      raw_class_spatial_attr(img, layer, tag.split(':')[-1].strip().split(",")[0], override=None),
                      raw_class_spatial_attr(img, layer, None, override=None),
                      clip=True
                    )
                    saliency_layers.append(saliency[0])
                    saliency_values.append(saliency[1])
                    imsave("../saliency_maps/" + file.split('/')[-1] + '_' + layer + ".png", saliency[0], plugin='pil', format_str='png')

                    attrs = raw_spatial_spatial_attr(img, layer, "mixed5b", override=None)
                    attrs = attrs / attrs.max()
                    break
                break

Mashedpotato
/home/w266ajh/Documents/top5gen/similar-images/Mashedpotato/cauliflower2.jpg
1: probability: 0.98, label: cauliflower


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


/home/w266ajh/Documents/top5gen/similar-images/Mashedpotato/cauliflower1.jpg
1: probability: 0.99, label: cauliflower
/home/w266ajh/Documents/top5gen/similar-images/Mashedpotato/cauliflower3.jpg
1: probability: 0.99, label: cauliflower
Stocking
/home/w266ajh/Documents/top5gen/similar-images/Stocking/mitten3.jpg
1: probability: 0.99, label: mitten
/home/w266ajh/Documents/top5gen/similar-images/Stocking/sock2.jpg
1: probability: 0.99, label: sock
/home/w266ajh/Documents/top5gen/similar-images/Stocking/mitten1.jpg
1: probability: 0.99, label: mitten
/home/w266ajh/Documents/top5gen/similar-images/Stocking/sock1.jpg
1: probability: 0.87, label: sock
/home/w266ajh/Documents/top5gen/similar-images/Stocking/sock3.jpg
1: probability: 1.00, label: sock
/home/w266ajh/Documents/top5gen/similar-images/Stocking/mitten2.jpg
1: probability: 0.89, label: mitten
Schoolbus
/home/w266ajh/Documents/top5gen/similar-images/Schoolbus/ambulance2.jpg
1: probability: 0.96, label: ambulance
/home/w266ajh/Document

In [ ]:
# SINGLE INPUT DIRS

# This block gets the saliency maps themselves

layers = ['mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e', 'mixed5a', 'mixed5b']


# Use for multiple input dirs
# Root dir
root_dir = "/home/w266ajh/Documents/top5gen/data_imageparts_1/"

# Manually iterate across image dir
for file in listdir(root_dir):
    if file.endswith(".jpg"):
        img = PIL.Image.open(root_dir + file)
        big_dim = max(img.width, img.height)
        wide = img.width > img.height
        new_w = 224 if not wide else int(img.width * 224 / img.height)
        new_h = 224 if wide else int(img.height * 224 / img.width)
        img = img.resize((new_w, new_h)).crop((0, 0, 224, 224))
        img = (np.asarray(img) / 255.0).astype(np.float32)
        cur_labels = get_top_five(root_dir + file, img)
    else: continue
    for label in cur_labels:
        # Note: Must save the file off each time because it will not re-read raw ndarray
        file = label.pop(0)
        print(file)
        img = resize(load(file), (224, 224))  # Make all 224x224
        imsave("current.jpeg", img, plugin='pil', format_str='jpeg')
        img = load("current.jpeg")
        for tag in label:
            print(tag)
            saliency_layers = []
            saliency_values = []
            for layer in layers:
                # Get our saliency layer
                saliency = orange_blue(
                  raw_class_spatial_attr(img, layer, tag.split(':')[-1].strip().split(",")[0], override=None),
                  raw_class_spatial_attr(img, layer, None, override=None),
                  clip=True
                )
                saliency_layers.append(saliency[0])
                saliency_values.append(saliency[1])
                imsave("../saliency_maps/" + file.split('/')[-1] + '_' + layer + ".png", saliency[0], plugin='pil', format_str='png')

                attrs = raw_spatial_spatial_attr(img, layer, "mixed5b", override=None)
                attrs = attrs / attrs.max()
                break
            break

/home/w266ajh/Documents/top5gen/data_imageparts_1/weevil2.jpg
1: probability: 0.98, label: weevil


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


/home/w266ajh/Documents/top5gen/data_imageparts_1/schoolbus4.jpg
1: probability: 1.00, label: school bus
/home/w266ajh/Documents/top5gen/data_imageparts_1/remote2.jpg
1: probability: 0.97, label: remote control, remote
/home/w266ajh/Documents/top5gen/data_imageparts_1/burger2.jpg
1: probability: 0.94, label: cheeseburger
/home/w266ajh/Documents/top5gen/data_imageparts_1/stocking5.jpg
1: probability: 1.00, label: Christmas stocking
/home/w266ajh/Documents/top5gen/data_imageparts_1/colobus4.jpg
1: probability: 0.85, label: colobus, colobus monkey
/home/w266ajh/Documents/top5gen/data_imageparts_1/train5.jpg
1: probability: 0.88, label: bullet train, bullet
/home/w266ajh/Documents/top5gen/data_imageparts_1/remote5.jpg
1: probability: 0.97, label: remote control, remote
/home/w266ajh/Documents/top5gen/data_imageparts_1/weevil3.jpg
1: probability: 0.98, label: weevil
/home/w266ajh/Documents/top5gen/data_imageparts_1/nematode3.jpg
1: probability: 1.00, label: nematode, nematode worm, roundwor

In [17]:
def highlight_interest_areas(saliency, input_image):
    # Load in our image
    img = np.array(Image.open(saliency).resize((224,224))) / 255  # Adjust input saliency here
    

    # Eliminate the low values--leave only high
    img[img<0.5] = 0  # Adjust the dropout here

    # Save off modified saliency
    imsave("testing.png", img, plugin='pil', format_str='png')

    ###

    # Get our overlay image
    img_pastable = Image.open(input_image)  # Adjust input image here
    img_pastable = img_pastable.resize((224, 224), Image.ANTIALIAS)
    img_overlay = Image.open("testing.png")
    img_overlay = img_overlay.resize((224, 224))
    img_overlay = img_overlay.convert("RGBA")
    datas = img_overlay.getdata()

    newData = []
    for item in datas:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            newData.append((0, 0, 0, 0))
        else:
            newData.append(item)

    img_overlay.putdata(newData)

    img_pastable.paste(img_overlay, (0,0), img_overlay)
    img_pastable.save('overlaid', 'png', quality=100)

    ###

    # Get the overlaid region out of the original image
    x, y, z = (img != 0).nonzero()

    # Iterate through the photo, using only the things ID'd as not highlighted
    photo_chunk = np.zeros((224,224,3))
    photo = np.array(Image.open(input_image).resize((224,224), Image.ANTIALIAS)) / 255	# Adjust input image here
    imsave("testing_chunks.png", photo, plugin='pil', format_str='png')

    for idx, subx in enumerate(x):
      for i in range(0, 3):
        photo_chunk[subx, y[idx], i] = photo[subx, y[idx], i]
    imsave("/home/w266ajh/Documents/output_interests/" + input_image.split('/')[-1], photo_chunk, plugin='pil', format_str='png')

In [ ]:
# MULTIPLE FILE PATHS

# Single file path
# root_img_path = "/home/w266ajh/Documents/top5gen/data_imageparts"

# Multiple file paths
# Root dir
root_img_path = "/home/w266ajh/Documents/top5gen/similar-images/"

# Go thorugh each dir in root dir
image_list = []
for dir in listdir(root_dir):
    go = False
    print(dir)
    # Continue if dir is empty
    if listdir(root_dir + dir) == list(): continue
    # Continue if no jpgs
    for file in listdir(root_dir + dir):
        if file.endswith(".jpg"): go = True
    if not go: continue
    image_list += [root_img_path + dir + '/' + file for file in listdir(root_dir + dir)
                    if file.endswith(".jpg")]
root_sal_path = "/home/w266ajh/Documents/saliency_maps"
# image_list = sorted([image for image in listdir(root_img_path) if image.endswith(".jpg")])
saliency_list = sorted([sal for sal in listdir(root_sal_path)])
for idx, img in enumerate(sorted(image_list, key=lambda path: path.split('/')[-1])):
    print(img)
    print(saliency_list[idx])
    highlight_interest_areas(root_sal_path + '/' + saliency_list[idx], img)

In [84]:
# SINGLE FILE PATH

# Single file path
root_img_path = "/home/w266ajh/Documents/top5gen/data_imageparts"
root_sal_path = "/home/w266ajh/Documents/saliency_maps"

image_list = sorted([image for image in listdir(root_img_path) if image.endswith(".jpg")])
saliency_list = sorted([sal for sal in listdir(root_sal_path)])
for idx, img in enumerate(sorted(image_list, key=lambda path: path.split('/')[-1])):
    print(img)
    print(saliency_list[idx])
    highlight_interest_areas(root_sal_path + '/' + saliency_list[idx], root_img_path + '/' + img)

barn1.jpg
barn1.jpg_mixed3a.png
barn2.jpg
barn2.jpg_mixed3a.png


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


barn3.jpg
barn3.jpg_mixed3a.png
barn4.jpg
barn4.jpg_mixed3a.png
barn5.jpg
barn5.jpg_mixed3a.png
dam1.jpg
dam1.jpg_mixed3a.png
dam2.jpg
dam2.jpg_mixed3a.png
dam3.jpg
dam3.jpg_mixed3a.png
dam4.jpg
dam4.jpg_mixed3a.png
dam5.jpg
dam5.jpg_mixed3a.png
fridge1.jpg
fridge1.jpg_mixed3a.png
fridge2.jpg
fridge2.jpg_mixed3a.png
fridge3.jpg
fridge3.jpg_mixed3a.png
fridge4.jpg
fridge4.jpg_mixed3a.png
fridge5.jpg
fridge5.jpg_mixed3a.png
gaspump1.jpg
gaspump1.jpg_mixed3a.png
gaspump2.jpg
gaspump2.jpg_mixed3a.png
gaspump3.jpg
gaspump3.jpg_mixed3a.png
gaspump4.jpg
gaspump4.jpg_mixed3a.png
gaspump5.jpg
gaspump5.jpg_mixed3a.png
mashedpotato1.jpg
mashedpotato1.jpg_mixed3a.png
mashedpotato2.jpg
mashedpotato2.jpg_mixed3a.png
mashedpotato3.jpg
mashedpotato3.jpg_mixed3a.png
mashedpotato4.jpg
mashedpotato4.jpg_mixed3a.png
mashedpotato5.jpg
mashedpotato5.jpg_mixed3a.png
pier1.jpg
pier1.jpg_mixed3a.png
pier2.jpg
pier2.jpg_mixed3a.png
pier3.jpg
pier3.jpg_mixed3a.png
pier4.jpg
pier4.jpg_mixed3a.png
pier5.jpg
pier5.

In [ ]:
getcwd()